In [1]:
import numpy as np
import numba
from numba import jit
import plotly.graph_objects as go
import plotly.io as pio
pio.templates.default = 'simple_white'

In [2]:
time = np.loadtxt('full_alpha_coherent_time.txt', unpack=True, dtype=float)
sync = np.loadtxt('full_alpha_coherent_sync.txt', unpack=True, dtype=float)

In [ ]:
total_time = time[time > 0][-1] 
total_time

In [ ]:
sync[0:15]

In [ ]:
lead_id = 3
lag_id = 1
    
lead_events = np.where(sync == lead_id, 1, 0)
lag_events = np.where(sync == lag_id, 1, 0)

In [ ]:
print('lead: ', lead_events[0:20])
print('lag : ', lag_events[0:20])

In [7]:
@jit(nopython=True)
def g2_unheralded(lead, lag, time, tau_arr, tau_bin_size, coincidence_window=5000):
    
    total_time = time[time > 0][-1]

    g2 = []

    for tau in tau_arr:

        coincidence_counter = 0
        
        for i, c in enumerate(lead[1:-1]):
            # i is the previous index

            # Skip if there is no event
            if c==0:
                continue
            
            # time of the event
            current_time = time[i+1]
            
            # time of the previous and next event
            previous_time = 0
            next_time = total_time+2*coincidence_window
            
            # check if there is an event in the lag channel one time step before or afer the current event
            if lag[i] == 1:
                previous_time = time[i] 
            if lag[i+2] == 1:
                next_time = time[i+2]

            # time difference between the events  
            dt_prev = previous_time - current_time 
            dt_next = next_time - current_time 
            
            # check if events occur in the coincidence window
            if abs(dt_next) <= coincidence_window:
                # check if events occur around time tau
                if  tau - tau_bin_size/2 <= dt_next <= tau + tau_bin_size/2:
                    coincidence_counter += 1 
            elif abs(dt_prev) <= coincidence_window:
                 # check if events occur around time tau
                if tau - tau_bin_size/2 <= dt_prev <= tau + tau_bin_size/2:
                    coincidence_counter += 1                 
            
        g2.append( coincidence_counter / ( lead.sum() * lag.sum() ) * total_time / coincidence_window * coincidence_window / tau_bin_size)
    
    return g2

In [8]:
tau_bin_size = 50
tau_max = 5000

tau = np.arange(-tau_max+tau_bin_size, tau_max, tau_bin_size)

In [9]:
g2 = g2_unheralded(lead_events, lag_events, time, tau_arr=tau, tau_bin_size=tau_bin_size)

In [15]:
fig = go.Figure()

fig.update_layout(
    title="Korrelationsfunktion 2. Ordnung (Kohärentes Licht)",
    xaxis_title=r'$\tau\ \text{[ps]}$',
    yaxis_title=r'$g_{2, \text{unheralded}}$',
    yaxis = dict(
        tickmode = 'linear',
        #tick0 = 0,
        #dtick = 0.1,
    )
)
fig.update_yaxes(rangemode="tozero")
fig.add_trace(go.Scatter(x=tau, y=g2))
fig

In [105]:
def g2_unheralded_old(sync, time, tau, tau_bin_size, coincidence_window=5000):
    
    total_time = time[time > 0][-1] - time[0]

    lead = 3
    lag = 1
    
    B1_events = np.where(sync == lead, 1, 0)
    B2_events = np.where(sync == lag, 1, 0)

    coincidence_counter = 0

    for i, c in enumerate(sync[1:-1]):
        # i is the previous index
            
        if sync[i] == c:
            continue

        previous_time = time[i]
        current_time = time[i+1]
        
        dt = current_time - previous_time
        
        if c == 1:
            dt = -dt
            
        if abs(dt) < abs(coincidence_window):
                        
            if tau - tau_bin_size/2 < dt < tau + tau_bin_size/2 :

                coincidence_counter += 1
    
    g2 = coincidence_counter # / ( sum(B1_events) * sum(B2_events) ) * total_time / coincidence_window
    
    return g2

In [ ]:
ls